In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
tf.__version__

In [ ]:
tf.test.is_gpu_available()

# Data Pre-Processing 

In [ ]:
# sets up the data generator with the required transformations
train_datagen = ImageDataGenerator(
    rescale=1./255,     # deafults from doc
    shear_range=0.2,    # deafults from doc
    zoom_range=0.2,      # deafults from doc
    horizontal_flip=True)

# Imports the training data set from the directories and resizes the images (for faster performance)
training_set = train_datagen.flow_from_directory(
    'dataset/training_set',     # path to training set
    target_size=(64, 64),       # size of the images pixels X pixels
    batch_size=32,              # 32 images per batch
    class_mode='binary')        # type of outcome binary or catagorical, since we have 2 options we use binary

## Pre processing test set

the test data must be rescaled as well in order to increase our prediction accuracy 

In [ ]:
# Creates an image data generator that will rescale our test set
test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_directory(
        'dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

In [56]:
test_set[61][0][31][5][1],test_set[61][1][31]

(array([0.5882353 , 0.5568628 , 0.48235297], dtype=float32), 0.0)

# Building the CNN

## Initilising the CNN

In [ ]:
cnn = tf.keras.models.Sequential()

## Step 1 Convolution

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

## Step 2 Pooling

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

## add a second conv and pooling layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation='relu'))


In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))


## step 3 flattening

In [ ]:
cnn.add(tf.keras.layers.Flatten())

## step 4 full connection

In [ ]:
cnn.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))

In [ ]:
cnn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Training the model

## Compiling the CNN

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Training on the training set then evaluating on the test set

In [ ]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 5)
# batch_size = Integer or None. Number of samples per gradient update. If unspecified, batch_size will default to 32. Do not specify the batch_size if your data is in the form of datasets, generators, or keras.utils.Sequence instances (since they generate batches).

In [ ]:
cnn.save('cnn_model')
new_model = tf.keras.models.load_model('cnn_model')

# Making a prediction

In [ ]:
import numpy as np
from tensorflow.keras.utils import load_img,img_to_array
test_image = load_img('dataset/single_prediction/cat_or_dog_4.jpg', target_size = (64, 64))
test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
test_image.shape

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_4.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
print(result[0][0])
prediction = ''
if result[0][0] == 1.0:
  prediction = 'dog'
else:
  prediction = 'cat'
print(prediction)


In [ ]:
print(np.shape(test_image))
#(batch / num of images, vertical pixels, horizontal pixels, RGB)

In [ ]:
[ (image)
    [ (vertical pixels)
        [ (horizontal pixels)
            [ (RGB)
                [255,255,255],
                [255,255,255],
                [255,255,255],
                ...image width = 64
            ]
        ], 
        [ 
            [
                [255,255,255],
                [255,255,255],
                [255,255,255],
                ...image width = 64
            ]
        ],
        ...image height = 64 
    ],
    [ (image 2)
        [ (vertical pixels)
            [ (horizontal pixels)
                [255,255,255],
                [255,255,255],
                [255,255,255],
                ...image width = 64
            ]
        ], 
        [ 
            [
                [255,255,255],
                [255,255,255],
                [255,255,255],
                ...image width = 64
            ]
        ],
        ...image height = 64 
    ],
    ... number of images per batch = 32     
]